# Reti Neurali con Python e Numpy

## Classe per una Rete Neurale
Il processo di creazione di questa rete neurale è descritto passo passo in [questo articolo sul blog di ProfessionAI](http://blog.profession.ai/creare-una-rete-neurale-artificiale-da-zero/)

In [1]:
import numpy as np

class NeuralNetwork:
  
  
  def __init__(self, hidden_layer_size=100):
    
    self.hidden_layer_size=hidden_layer_size
    
    
  def _init_weights(self, input_size, hidden_size):
    
    self._W1 = np.random.randn(input_size, hidden_size)
    self._b1 = np.zeros(hidden_size)
    self._W2 = np.random.randn(hidden_size,1)
    self._b2 = np.zeros(1)

    
  def _accuracy(self, y, y_pred):      
    return np.sum(y==y_pred)/len(y)
  
  
  def _log_loss(self, y_true, y_proba):
    return -np.sum(np.multiply(y_true,np.log(y_proba))+np.multiply((1-y_true),np.log(1-y_proba)))/len(y_true)
  
  
  def _relu(self, Z):
    return np.maximum(Z, 0)

  
  def _sigmoid(self, Z):
    return 1/(1+np.power(np.e,-Z))
  
  
  def _relu_derivative(self, Z):
    dZ = np.zeros(Z.shape)
    dZ[Z>0] = 1
    return dZ
    
               
  def _forward_propagation(self, X):
                     
    Z1 = np.dot(X,self._W1)+self._b1

    A1 = self._relu(Z1)
    Z2 = np.dot(A1,self._W2)+self._b2
    A2 = self._sigmoid(Z2)
    
    self._forward_cache = (Z1, A1, Z2, A2)

    return A2.ravel()


  def predict(self, X, return_proba=False):

      proba = self._forward_propagation(X)

      y = np.zeros(X.shape[0])
      y[proba>=0.5]=1
      y[proba<0.5]=0

      if(return_proba):
        return (y, proba)
      else:
        return proba
                            
      
  def _back_propagation(self, X, y):
  
    Z1, A1, Z2, A2 = self._forward_cache
                   
    m = A1.shape[1]
    
    dZ2 = A2-y.reshape(-1,1)
    dW2 = np.dot(A1.T, dZ2)/m
    db2 = np.sum(dZ2, axis=0)/m

    dZ1 = np.dot(dZ2, self._W2.T)*self._relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)/m
    db1 = np.sum(dZ1, axis=0)/m
    
    return dW1, db1, dW2, db2
           
               
  def fit(self, X, y, epochs=200, lr=0.01):
     
    self._init_weights(X.shape[1], self.hidden_layer_size)
      
    for _ in range(epochs):
      Y = self._forward_propagation(X)
      dW1, db1, dW2, db2 = self._back_propagation(X, y)
      self._W1-=lr*dW1
      self._b1-=lr*db1
      self._W2-=lr*dW2
      self._b2-=lr*db2
               

  def evaluate(self, X, y):
    y_pred, proba = self.predict(X, return_proba=True)
    accuracy = self._accuracy(y, y_pred)
    log_loss = self._log_loss(y, proba)
    return (accuracy, log_loss)

## Testiamo la Rete Neurale
Per testare la nostra Rete Neurale utilizzeremo il Boston Breast Cancer Dataset, un dataset di tumori al seno etichettati come positivi o negativi.
Importiamo il dataset direttamente dalla Repository Github dei Tutorial di ProfessionAI, per farlo possiamo utilizzare Pandas, una popolare libreria Python per l’analisi dati.

In [2]:
import pandas as pd

CSV_URL = "https://raw.githubusercontent.com/ProfAI/tutorials/master/Come%20Creare%20una%20Rete%20Neurale%20da%20Zero/breast_cancer.csv"

breast_cancer = pd.read_csv(CSV_URL)

Il risultato sarà un DataFrame, una struttura dati che Pandas usa per rappresentare dati tabulari, possiamo avere una preview del suo contenuto usando il metodo .head().<br>
Il nostro dataset contiene in totale 563 righe (e quindi esempi) e 31 colonne, cioè 30 features e un target, che è la colonna “malignant”.<br>
Estraiamo features e target in array numpy.

In [3]:
X = breast_cancer.drop("malignant", axis=1).values
y = breast_cancer["malignant"].values

Ora dobbiamo dividere ogni array in due array distinti, uno per l’addestramento e uno per il test. Questa divisione serve per poter verificare le reali capacità predittive del modello, testandolo su dati che non ha già visto durante la fase di addestramento.
Creiamo una funzione train_test_split per eseguire questa divisione:

In [4]:
def train_test_split(X, y, test_size=0.3, random_state=None):

  if(random_state!=None):
    np.random.seed(random_state)
  
  n = X.shape[0]

  test_indices = np.random.choice(n, int(n*test_size), replace=False) # selezioniamo gli indici degli esempi per il test set
  
  # estraiamo gli esempi del test set
  # in base agli indici
  
  X_test = X[test_indices]
  y_test = y[test_indices]
  
  # creiamo il train set
  # rimuovendo gli esempi del test set
  # in base agli indici
  
  X_train = np.delete(X, test_indices, axis=0)
  y_train = np.delete(y, test_indices, axis=0)

  return (X_train, X_test, y_train, y_test )


X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.3)

Abbiamo assegnato il 30% degli esempi del dataset al set di test, quindi abbiamo 395 esempi per l’addestramento e 168 per il test, sono un po’ pochi per l’addestramento di una rete neurale, ma trattandosi di un modello con un solo strato nascosto possono andare bene.
E’ buona norma portare le features in un range di valori comune, questo può velocizzare anche di tanto la fase di addestramento.
Utilizziamo la normalizzazione, che si esegue sottraendo il valore minore e dividendo per la differenza tra il valore maggiore e il valore minore:

$$ X_{norm} = \frac{X-X_{min}}{X_{max}-X_{min}} $$

Ricorda che dobbiamo sempre applicare le stesse trasformazioni ai dati di addestramento, a quelli di test, e in generale a tutti quelli che daremo in pasto alla nostra rete neurale, quindi calcoliamo massimo e minimo sul set di addestramento e usiamo questi valori per la normalizzazione di entrambi gli array.

In [5]:
X_max = X_train.max(axis=0)
X_min = X_train.min(axis=0)

X_train = (X_train - X_min)/(X_max-X_min)
X_test = (X_test - X_min)/(X_max-X_min)

Perfetto ! Adesso creiamo la nostra rete con 10 nodi sullo strato nascosto, addestriamola sul set di addestramento per 500 epoche e valutiamola sul set di test.

In [6]:
model = NeuralNetwork()
model.fit(X_train, y_train, epochs=500, lr=0.01)
model.evaluate(X_test, y_test)

(0.9464285714285714, 0.210132727184693)

Dato che i pesi vengono inizializzati a valori casuali il risultato può lievemente variare tra diverse esecuzioni della rete.

## La Rete Neurale funziona. E ora ?
Mettiamo caso di ricevere i risultati di 6 nuovi esami radiografici, le features estratte da questi ci vengono consegnate all’interno di un file csv, carichiamolo con pandas, estraiamolo le features e normalizziamole

In [7]:
exams_df = pd.read_csv("https://raw.githubusercontent.com/ProfAI/tutorials/master/Come%20Creare%20una%20Rete%20Neurale%20da%20Zero/exam%20results.csv")

X_new = exams_df.values
X_new = (X_new - X_min)/(X_max-X_min)

Ora utilizziamo il metodo predict per classificare i risultati di tali esami, in modo da identificare eventuali tumori maligni, ottenendo anche la probabilità.

In [8]:
y_pred, y_proba = model.predict(X_new, return_proba=True)

Stampiamo il risultato

In [9]:
classes = ["benigno", "maligno"]

for i, (pred, proba) in enumerate(zip(y_pred, y_proba)):
  print("Risultato %d = %s (%.4f)" % (i+1, classes[int(pred)], proba))

Risultato 1 = benigno (0.0000)
Risultato 2 = maligno (0.9984)
Risultato 3 = maligno (0.9959)
Risultato 4 = benigno (0.0043)
Risultato 5 = benigno (0.4778)
Risultato 6 = benigno (0.0238)


Quando la probabilità associata non è alta andrebbero eseguiti ulteriori esami di verifica, specialmente nel caso di un tumore classificato come benigno, dato che classificare erroneamente un tumore maligno come benigno è molto più grave che classificare un tumore benigno come maligno.
Come abbiamo visto in questo tutorial, in questi casi è opportuno utilizzare anche la matrice di confusione come metrica per valutare il modello.